In [1]:

import pandas as pd


In [2]:
df_betting = pd.read_csv('data/bettingmarkets/betting_data.csv')

df_betting.head()


,Outcome,Implied Probability (%),Market Description,Data Source
0,Oklahoma City Thunder,38.0,Championship Winner,Polymarket
1,Boston Celtics,29.0,Championship Winner,Polymarket
2,Cleveland Cavaliers,16.0,Championship Winner,Polymarket
3,Golden State Warriors,6.0,Championship Winner,Polymarket
4,Minnesota Timberwolves,4.0,Championship Winner,Polymarket


In [3]:
# 1. Convert implied probabilities (%) → fraction [0,1]
df_betting['ImpProb'] = df_betting['Implied Probability (%)'] / 100.0

# 2. Compute overround per Data Source & Market Description
overround = (
    df_betting
    .groupby(['Data Source', 'Market Description'])['ImpProb']
    .sum()
    .reset_index(name='Overround')
)

# 3. Merge the overround back into the main DataFrame
df_betting = df_betting.merge(overround, on=['Data Source', 'Market Description'])

# 4. Remove vig by normalizing: divide by the overround
df_betting['NormProb'] = df_betting['ImpProb'] / df_betting['Overround']




In [4]:
verification = (
    df_betting
    .groupby(['Data Source', 'Market Description'])['NormProb']
    .sum()
    .reset_index(name='Sum')
)

print("\nVerification that normalization worked:")
print(verification.head(10))




Verification that normalization worked:
  Data Source         Market Description  Sum
0   Bet365 US  Eastern Conference Winner  1.0
1   Bet365 US  Western Conference Winner  1.0
2      BetMGM  Eastern Conference Winner  1.0
3      BetMGM  Western Conference Winner  1.0
4   BetRivers        Championship Winner  1.0
5   BetRivers  Eastern Conference Winner  1.0
6   BetRivers  Western Conference Winner  1.0
7     Borgata        Championship Winner  1.0
8     Borgata  Eastern Conference Winner  1.0
9     Borgata  Western Conference Winner  1.0


In [5]:
df_betting.head(10)

,Outcome,Implied Probability (%),Market Description,Data Source,ImpProb,Overround,NormProb
0,Oklahoma City Thunder,38.0,Championship Winner,Polymarket,0.38,1.02,0.372549
1,Boston Celtics,29.0,Championship Winner,Polymarket,0.29,1.02,0.284314
2,Cleveland Cavaliers,16.0,Championship Winner,Polymarket,0.16,1.02,0.156863
3,Golden State Warriors,6.0,Championship Winner,Polymarket,0.06,1.02,0.058824
4,Minnesota Timberwolves,4.0,Championship Winner,Polymarket,0.04,1.02,0.039216
5,Denver Nuggets,3.0,Championship Winner,Polymarket,0.03,1.02,0.029412
6,Los Angeles Lakers,2.0,Championship Winner,Polymarket,0.02,1.02,0.019608
7,Los Angeles Clippers,2.0,Championship Winner,Polymarket,0.02,1.02,0.019608
8,New York Knicks,1.0,Championship Winner,Polymarket,0.01,1.02,0.009804
9,Indiana Pacers,1.0,Championship Winner,Polymarket,0.01,1.02,0.009804


# Elo and Bardley terry

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge

# --- 1. Load and re-index the head-to-head matrix ---
df_h2h = pd.read_csv('teamVsteam2025.csv')

# Derive mapping from full team names to their 3-letter codes
full_names = df_h2h['Team']
abbrs = []
for idx, full in enumerate(full_names):
    row = df_h2h.iloc[idx, 2:]  # Skip 'Rk' and 'Team'
    self_col = row[row.isna() | (row == '')].index
    if len(self_col) != 1:
        raise ValueError(f"Can't identify the abbreviation column for {full}")
    abbrs.append(self_col[0])

mapping = dict(zip(full_names, abbrs))

# Apply mapping: set index to 3-letter codes, drop 'Rk'
df_h2h = df_h2h.set_index('Team').drop(columns=['Rk'])
df_h2h.index = df_h2h.index.map(mapping)

teams = df_h2h.index.tolist()

# --- 2. Expand aggregate W-L records into individual game rows ---
matches = []
for i, team_i in enumerate(teams):
    for j, team_j in enumerate(teams):
        if i >= j:
            continue
        record = df_h2h.at[team_i, team_j]
        if pd.isna(record) or record == '':
            continue
        try:
            wins, losses = map(int, record.split('-'))
            matches += [{'winner': team_i, 'loser': team_j}] * wins
            matches += [{'winner': team_j, 'loser': team_i}] * losses
        except (ValueError, AttributeError):
            print(f"Warning: Could not parse record '{record}' between {team_i} and {team_j}")

# Create DataFrame from the matches list we've built
df_matches = pd.DataFrame(matches)

# --- 3. Build Basic Bradley-Terry model ---
if not matches:
    print("No valid matches found. Check the format of your head-to-head data.")
else:
    # Load standings data for additional features
    try:
        df_standings = pd.read_csv('standings2025.csv')
        print(f"Loaded standings data with {len(df_standings)} teams")
        
        # Process standings data to extract useful features
        # We'll set this up for later use in the enhanced model
        standings_features = {}
        
    except Exception as e:
        print(f"Could not load standings data: {e}")
        df_standings = None
    
    # Create team strength parameters for each team
    team_indices = {team: i for i, team in enumerate(teams)}
    n_teams = len(teams)
    
    # Set up the design matrix where each row represents a game
    X = np.zeros((len(df_matches), n_teams))
    
    for i, (_, match) in enumerate(df_matches.iterrows()):
        winner = match['winner']
        loser = match['loser']
        winner_idx = team_indices[winner]
        loser_idx = team_indices[loser]
        
        # For Bradley-Terry, we code winner as +1, loser as -1
        X[i, winner_idx] = 1
        X[i, loser_idx] = -1
    
    # Set up the target: all 1's for observation = "winner won"
    y = np.ones(len(df_matches))
    
    # Use Ridge regression with small alpha for numerical stability
    bt_model = Ridge(alpha=0.001, fit_intercept=False)
    bt_model.fit(X, y)
    
    # Extract team strengths
    abilities = pd.Series(bt_model.coef_, index=teams)
    
    # Rescale for better interpretability (0-100 scale)
    min_ability = abilities.min()
    max_ability = abilities.max()
    scaled_abilities = (abilities - min_ability) * 100 / (max_ability - min_ability)
    
    # Calculate expected win probabilities
    def win_probability(team1, team2):
        s1 = abilities[team1]
        s2 = abilities[team2]
        return 1 / (1 + np.exp(-(s1 - s2)))
    
    # Display the top teams
    print("\nBradley-Terry model results:")
    print("============================")
    print("\nTop 10 Teams by Bradley-Terry ability (0-100 scale):")
    top_teams = scaled_abilities.sort_values(ascending=False).head(10)
    print(top_teams)
    
    # Show some probabilities for top matchups
    print("\nPredicted win probabilities for key matchups:")
    for i, team1 in enumerate(top_teams.index[:3]):
        for team2 in top_teams.index[i+1:4]:
            prob = win_probability(team1, team2)
            print(f"{team1} vs {team2}: {team1} has {prob:.1%} chance to win")
    
    # Calculate team win percentages
    win_counts = df_matches['winner'].value_counts()
    loss_counts = df_matches['loser'].value_counts()
    
    win_pct = {}
    for team in teams:
        wins = win_counts.get(team, 0)
        losses = loss_counts.get(team, 0)
        total = wins + losses
        win_pct[team] = wins / total if total > 0 else 0
    
    win_pct_series = pd.Series(win_pct)
    
    print("\nTop 10 Teams by Win Percentage:")
    print(win_pct_series.sort_values(ascending=False).head(10))
    
    # Create Elo ratings for comparison
    K = 20
    elo_ratings = {team: 1500 for team in teams}
    for _, row in df_matches.iterrows():
        w, l = row['winner'], row['loser']
        R_w, R_l = elo_ratings[w], elo_ratings[l]
        p_win = 1 / (1 + 10 ** ((R_l - R_w) / 400))
        elo_ratings[w] += K * (1 - p_win)
        elo_ratings[l] += K * (0 - (1 - p_win))
    
    elo_series = pd.Series(elo_ratings)
    
    print("\nTop 10 Teams by Elo rating:")
    print(elo_series.sort_values(ascending=False).head(10))
    
    # Correlate the different metrics
    print("\nCorrelation between metrics:")
    metrics = pd.DataFrame({
        'BT_ability': scaled_abilities,
        'Elo_rating': elo_series,
        'Win_pct': win_pct_series
    })
    print(metrics.corr())
    
    # Output final team ratings for use in step 4
    team_ratings = pd.DataFrame({
        'BT_ability': scaled_abilities,
        'Elo_rating': elo_series,
        'Win_pct': win_pct_series
    })
    
    print("\nFinal team ratings (for use in step 4):")
    print(team_ratings.sort_values('BT_ability', ascending=False).head(10))
    
    

Loaded standings data with 31 teams

Bradley-Terry model results:

Top 10 Teams by Bradley-Terry ability (0-100 scale):
OKC    100.000000
CLE     89.641168
BOS     82.951806
HOU     69.808000
LAC     64.960128
LAL     64.335839
DEN     64.314075
NYK     64.061554
MIN     62.909234
IND     61.533913
dtype: float64

Predicted win probabilities for key matchups:
OKC vs CLE: OKC has 53.1% chance to win
OKC vs BOS: OKC has 55.1% chance to win
OKC vs HOU: OKC has 58.9% chance to win
CLE vs BOS: CLE has 52.0% chance to win
CLE vs HOU: CLE has 55.9% chance to win
BOS vs HOU: BOS has 53.9% chance to win

Top 10 Teams by Win Percentage:
OKC    0.829268
CLE    0.780488
BOS    0.743902
HOU    0.634146
NYK    0.621951
LAC    0.609756
DEN    0.609756
LAL    0.609756
IND    0.609756
MIN    0.597561
dtype: float64

Top 10 Teams by Elo rating:
OKC    1732.728374
CLE    1696.743593
BOS    1676.735376
NYK    1610.170146
LAL    1589.728038
HOU    1588.396023
MEM    1587.402879
MIL    1579.261894
LAC    15

In [7]:
import pandas as pd
import numpy as np
from math import comb

# ----- Step 4: Convert Game Odds → Series Odds -----

def calc_series_win_probability(p, wins_needed=4):
    """
    Calculate the probability of winning a best-of-(2*wins_needed-1) series
    where a team needs wins_needed wins to win the series.
    
    Args:
        p: Probability of winning a single game
        wins_needed: Number of wins needed to win the series (default: 4 for best-of-7)
    
    Returns:
        Probability of winning the series
    """
    if p <= 0.0:
        return 0.0
    if p >= 1.0:
        return 1.0
        
    q = 1 - p
    total = 0.0
    
    # Sum over number of losses k = 0..(wins_needed-1)
    for k in range(wins_needed):
        total += comb(wins_needed - 1 + k, k) * (p**wins_needed) * (q**k)
        
    return total

# Display table of game win vs series win probabilities
print("Step 4: Convert Game Odds → Series Odds")
print("======================================")

# Create reference table
p_game_values = np.linspace(0.5, 1.0, 11)
p_series_values = [calc_series_win_probability(p) for p in p_game_values]

probability_table = pd.DataFrame({
    'Game Win Probability': p_game_values,
    'Series Win Probability': p_series_values
})

print("\nRelationship between game win probability and series win probability:")
print(probability_table)

# Calculate pairwise matchup probabilities for all potential playoff teams
print("\nExample Series Win Probabilities for Top Teams:")
print("----------------------------------------------")

# Get the top 8 teams for playoff bracket simulation
top_teams = team_ratings.sort_values('BT_ability', ascending=False).head(8).index

# Create matrices to store game and series probabilities
game_probs = {
    'BT_ability': pd.DataFrame(index=top_teams, columns=top_teams),
    'Elo_rating': pd.DataFrame(index=top_teams, columns=top_teams),
    'Win_pct': pd.DataFrame(index=top_teams, columns=top_teams)
}

series_probs = {
    'BT_ability': pd.DataFrame(index=top_teams, columns=top_teams),
    'Elo_rating': pd.DataFrame(index=top_teams, columns=top_teams),
    'Win_pct': pd.DataFrame(index=top_teams, columns=top_teams)
}

# Show example matchups with all three models
top_4 = top_teams[:4]
for model in ['BT_ability', 'Elo_rating', 'Win_pct']:
    print(f"\n{model} model predictions:")
    for i, team1 in enumerate(top_4):
        for j, team2 in enumerate(top_4):
            if i >= j:  # Skip duplicate matchups
                continue
            
            # Calculate game win probability based on model
            if model == 'BT_ability':
                # Use a much smaller scaling factor for more reasonable probabilities
                # 0.02 should give probabilities in the 55-65% range for most matchups
                diff = team_ratings.loc[team1, 'BT_ability'] - team_ratings.loc[team2, 'BT_ability']
                p_game = 1 / (1 + np.exp(-diff * 0.02))
            elif model == 'Elo_rating':
                R1 = team_ratings.loc[team1, 'Elo_rating']
                R2 = team_ratings.loc[team2, 'Elo_rating']
                p_game = 1 / (1 + 10 ** ((R2 - R1) / 400))
            else:  # Win_pct
                w1 = team_ratings.loc[team1, 'Win_pct']
                w2 = team_ratings.loc[team2, 'Win_pct']
                w1_adj = 0.5 + (w1 - 0.5) * 0.67
                w2_adj = 0.5 + (w2 - 0.5) * 0.67
                p_game = w1_adj / (w1_adj + w2_adj) if (w1_adj + w2_adj) > 0 else 0.5
            
            # Calculate series win probability
            p_series = calc_series_win_probability(p_game)
            
            # Store probabilities in matrices
            game_probs[model].at[team1, team2] = p_game
            series_probs[model].at[team1, team2] = p_series
            
            # Print results
            print(f"{team1} vs {team2}: ")
            print(f"  - {team1} has {p_game:.1%} chance to win a single game")
            print(f"  - {team1} has {p_series:.1%} chance to win a best-of-7 series")

# Fill in all remaining matchups
for i, team1 in enumerate(top_teams):
    for j, team2 in enumerate(top_teams):
        if i == j or (team1 in top_4 and team2 in top_4 and i < j):
            continue  # Skip self-matchups and already calculated matchups
            
        # BT_ability with much reduced scaling factor
        diff = team_ratings.loc[team1, 'BT_ability'] - team_ratings.loc[team2, 'BT_ability']
        bt_p_game = 1 / (1 + np.exp(-diff * 0.02))
        game_probs['BT_ability'].at[team1, team2] = bt_p_game
        series_probs['BT_ability'].at[team1, team2] = calc_series_win_probability(bt_p_game)
        
        # Elo_rating
        R1 = team_ratings.loc[team1, 'Elo_rating']
        R2 = team_ratings.loc[team2, 'Elo_rating']
        elo_p_game = 1 / (1 + 10 ** ((R2 - R1) / 400))
        game_probs['Elo_rating'].at[team1, team2] = elo_p_game
        series_probs['Elo_rating'].at[team1, team2] = calc_series_win_probability(elo_p_game)
        
        # Win_pct
        w1 = team_ratings.loc[team1, 'Win_pct']
        w2 = team_ratings.loc[team2, 'Win_pct']
        w1_adj = 0.5 + (w1 - 0.5) * 0.67
        w2_adj = 0.5 + (w2 - 0.5) * 0.67
        wp_p_game = w1_adj / (w1_adj + w2_adj) if (w1_adj + w2_adj) > 0 else 0.5
        game_probs['Win_pct'].at[team1, team2] = wp_p_game
        series_probs['Win_pct'].at[team1, team2] = calc_series_win_probability(wp_p_game)

print("\nSummary of game win probability ranges:")
for model in game_probs:
    non_zero_probs = game_probs[model][game_probs[model] > 0]
    min_prob = non_zero_probs.min().min() if not non_zero_probs.empty else 0
    max_prob = non_zero_probs.max().max() if not non_zero_probs.empty else 0
    print(f"{model}: {min_prob:.1%} to {max_prob:.1%}")

print("\nStep 4 Complete: Converted Game Odds to Series Odds")
print("Ready to proceed to Step 5: Construct the Actual Playoff Bracket")

Step 4: Convert Game Odds → Series Odds

Relationship between game win probability and series win probability:
    Game Win Probability  Series Win Probability
0                   0.50                0.500000
1                   0.55                0.608288
2                   0.60                0.710208
3                   0.65                0.800154
4                   0.70                0.873964
5                   0.75                0.929443
6                   0.80                0.966656
7                   0.85                0.987897
8                   0.90                0.997272
9                   0.95                0.999806
10                  1.00                1.000000

Example Series Win Probabilities for Top Teams:
----------------------------------------------

BT_ability model predictions:
OKC vs CLE: 
  - OKC has 55.2% chance to win a single game
  - OKC has 61.2% chance to win a best-of-7 series
OKC vs BOS: 
  - OKC has 58.4% chance to win a single game
  - O

In [8]:
import pandas as pd
import numpy as np
import random
from math import comb
from scipy.stats import binom

# For reproducibility
random.seed(42)
np.random.seed(42)

# ----- Step 5: Construct the Actual Playoff Bracket -----

# Define conference and team seed information
# Western Conference teams with their seeds
west_seeds = {
    1: 'OKC',  # Thunder
    2: 'HOU',  # Rockets
    3: 'LAL',  # Lakers
    4: 'DEN',  # Nuggets
    5: 'LAC',  # Clippers
    6: 'MIN',  # Timberwolves
    7: 'GSW',  # Warriors
    8: 'MEM'   # Grizzlies
}

# Eastern Conference teams with their seeds
east_seeds = {
    1: 'CLE',  # Cavaliers
    2: 'BOS',  # Celtics
    3: 'NYK',  # Knicks
    4: 'IND',  # Pacers
    5: 'MIL',  # Bucks
    6: 'DET',  # Pistons
    7: 'ORL',  # Magic
    8: 'MIA'   # Heat
}

# Reverse mappings for convenience
west_teams = {team: seed for seed, team in west_seeds.items()}
east_teams = {team: seed for seed, team in east_seeds.items()}

# Current playoff status (based on the bracket)
current_status = {
    'west_r1': {
        (1, 8): ('OKC', 'MEM', 4, 0, 'complete'),      # Thunder won 4-0
        (4, 5): ('DEN', 'LAC', 3, 2, 'in_progress'),   # Denver leads 3-2
        (3, 6): ('LAL', 'MIN', 1, 4, 'complete'),      # Timberwolves won 4-1
        (2, 7): ('HOU', 'GSW', 3, 2, 'in_progress')    # Rockets lead 3-2
    },
    'east_r1': {
        (1, 8): ('CLE', 'MIA', 4, 0, 'complete'),      # Cavaliers won 4-0
        (4, 5): ('IND', 'MIL', 4, 1, 'complete'),      # Pacers won 4-1
        (3, 6): ('NYK', 'DET', 3, 2, 'in_progress'),   # Knicks lead 3-2
        (2, 7): ('BOS', 'ORL', 4, 1, 'complete')       # Celtics won 4-1
    }
}

# Display the current bracket
print("Current NBA Playoff Status:")
print("===========================")

print("\nWestern Conference - First Round:")
for seeds, (team1, team2, score1, score2, status) in current_status['west_r1'].items():
    seed1, seed2 = seeds
    if status == 'complete':
        print(f"({seed1}) {team1} def. ({seed2}) {team2}, {score1}-{score2}")
    elif status == 'in_progress':
        print(f"({seed1}) {team1} leads ({seed2}) {team2}, {score1}-{score2}")

print("\nEastern Conference - First Round:")
for seeds, (team1, team2, score1, score2, status) in current_status['east_r1'].items():
    seed1, seed2 = seeds
    if status == 'complete':
        print(f"({seed1}) {team1} def. ({seed2}) {team2}, {score1}-{score2}")
    elif status == 'in_progress':
        print(f"({seed1}) {team1} leads ({seed2}) {team2}, {score1}-{score2}")

# ----- Generate Game and Series Probabilities from team_ratings -----

# Create game probability matrices
game_probs = {}

# BT_ability game probabilities
game_probs['BT_ability'] = pd.DataFrame(index=team_ratings.index, columns=team_ratings.index)
for team1 in team_ratings.index:
    for team2 in team_ratings.index:
        if team1 == team2:
            game_probs['BT_ability'].at[team1, team2] = np.nan
        else:
            # Use the win_probability function from your original code
            s1 = abilities[team1]
            s2 = abilities[team2]
            prob = 1 / (1 + np.exp(-(s1 - s2)))
            game_probs['BT_ability'].at[team1, team2] = prob

# Elo_rating game probabilities
game_probs['Elo_rating'] = pd.DataFrame(index=team_ratings.index, columns=team_ratings.index)
for team1 in team_ratings.index:
    for team2 in team_ratings.index:
        if team1 == team2:
            game_probs['Elo_rating'].at[team1, team2] = np.nan
        else:
            # Use Elo formula
            R1 = team_ratings.loc[team1, 'Elo_rating']
            R2 = team_ratings.loc[team2, 'Elo_rating']
            prob = 1 / (1 + 10 ** ((R2 - R1) / 400))
            game_probs['Elo_rating'].at[team1, team2] = prob

# Win_pct game probabilities
game_probs['Win_pct'] = pd.DataFrame(index=team_ratings.index, columns=team_ratings.index)
for team1 in team_ratings.index:
    for team2 in team_ratings.index:
        if team1 == team2:
            game_probs['Win_pct'].at[team1, team2] = np.nan
        else:
            # Simple win percentage adjustment
            w1 = team_ratings.loc[team1, 'Win_pct']
            w2 = team_ratings.loc[team2, 'Win_pct']
            w1_adj = 0.5 + (w1 - 0.5) * 0.67
            w2_adj = 0.5 + (w2 - 0.5) * 0.67
            prob = w1_adj / (w1_adj + w2_adj) if (w1_adj + w2_adj) > 0 else 0.5
            game_probs['Win_pct'].at[team1, team2] = prob

# Function to calculate series win probability
def calc_series_win_probability(p, wins_needed=4):
    """
    Calculate the probability of winning a best-of-(2*wins_needed-1) series
    where a team needs wins_needed wins to win the series.
    """
    if p <= 0.0:
        return 0.0
    if p >= 1.0:
        return 1.0
        
    q = 1 - p
    total = 0.0
    
    # Sum over number of losses k = 0..(wins_needed-1)
    for k in range(wins_needed):
        total += comb(wins_needed - 1 + k, k) * (p**wins_needed) * (q**k)
        
    return total

# Create series probability matrices
series_probs = {}
for model in ['BT_ability', 'Elo_rating', 'Win_pct']:
    series_probs[model] = pd.DataFrame(index=team_ratings.index, columns=team_ratings.index)
    for team1 in team_ratings.index:
        for team2 in team_ratings.index:
            if team1 == team2:
                series_probs[model].at[team1, team2] = np.nan
            else:
                p_game = game_probs[model].at[team1, team2]
                p_series = calc_series_win_probability(p_game)
                series_probs[model].at[team1, team2] = p_series

# ----- Step 6: Monte Carlo Simulation of the Playoffs -----

def get_game_prob(team1, team2, model):
    """
    Get the probability of team1 winning a single game against team2
    using the specified model
    """
    # Check if we have this matchup in our probability matrix
    if team1 in game_probs[model].index and team2 in game_probs[model].columns:
        if pd.notna(game_probs[model].at[team1, team2]):
            return game_probs[model].at[team1, team2]
    
    # Check reverse direction
    if team2 in game_probs[model].index and team1 in game_probs[model].columns:
        if pd.notna(game_probs[model].at[team2, team1]):
            return 1 - game_probs[model].at[team2, team1]
    
    # Fallback: estimate based on seed difference
    seed1 = west_teams.get(team1, east_teams.get(team1, 16))
    seed2 = west_teams.get(team2, east_teams.get(team2, 16))
    
    # Higher seed advantage
    seed_diff = seed2 - seed1  # Positive if team1 is higher seed
    
    base_prob = 0.5
    if model == 'BT_ability':
        # Clamp between 0 and 1 as suggested
        return min(1, max(0, base_prob + 0.025 * seed_diff))
    elif model == 'Elo_rating':
        return min(1, max(0, base_prob + 0.03 * seed_diff))
    else:  # Win_pct
        return min(1, max(0, base_prob + 0.02 * seed_diff))

def get_series_prob(team1, team2, model):
    """
    Get the probability of team1 winning a 7-game series against team2
    """
    # Check if we have this matchup in our probability matrix
    if team1 in series_probs[model].index and team2 in series_probs[model].columns:
        if pd.notna(series_probs[model].at[team1, team2]):
            return series_probs[model].at[team1, team2]
    
    # Check reverse direction
    if team2 in series_probs[model].index and team1 in series_probs[model].columns:
        if pd.notna(series_probs[model].at[team2, team1]):
            return 1 - series_probs[model].at[team2, team1]
    
    # If not found, calculate from game probability
    p_game = get_game_prob(team1, team2, model)
    return calc_series_win_probability(p_game)

def simulate_in_progress_series(team1, team2, score1, score2, model):
    """
    Simulate the remainder of an in-progress series
    
    Args:
        team1, team2: Team codes
        score1, score2: Current series score (team1, team2)
        model: Which model to use
        
    Returns:
        Winner of the series
    """
    # Get game probability
    p_game = get_game_prob(team1, team2, model)
    
    # Calculate games remaining and wins needed
    games_played = score1 + score2
    games_left = 7 - games_played
    wins_needed_team1 = 4 - score1
    wins_needed_team2 = 4 - score2
    
    # If either team has already won, return that team
    if score1 >= 4:
        return team1
    if score2 >= 4:
        return team2
    
    # Probability that team1 wins the series from current state
    # This is 1 minus probability that team1 wins fewer than needed games
    p_team1_wins = 1 - binom.cdf(wins_needed_team1 - 1, games_left, p_game)
    
    # Simulate the outcome
    if random.random() < p_team1_wins:
        return team1
    else:
        return team2

def simulate_remaining_playoffs(n_simulations=10000):
    """
    Run Monte Carlo simulation of the NBA playoffs from current state
    
    Args:
        n_simulations: Number of simulations to run
        
    Returns:
        Dictionary with simulation results by model and ensemble
    """
    # For storing results by model
    all_results = {}
    
    # Get all teams
    all_teams = list(set(list(west_teams.keys()) + list(east_teams.keys())))
    
    # For each model (BT_ability, Elo_rating, Win_pct)
    for model in ['BT_ability', 'Elo_rating', 'Win_pct']:
        print(f"\nRunning {n_simulations} simulations using {model} model...")
        
        # Initialize counters
        champion_counts = {team: 0 for team in all_teams}
        conf_champs = {
            'West': {team: 0 for team in west_teams.keys()},
            'East': {team: 0 for team in east_teams.keys()}
        }
        finals_apps = {team: 0 for team in all_teams}
        
        # Run simulations
        for sim in range(n_simulations):
            # First round winners by seed matchup
            r1_winners = {'West': {}, 'East': {}}
            
            # Simulate remaining Western Conference first round
            for seeds, (team1, team2, score1, score2, status) in current_status['west_r1'].items():
                if status == 'complete':
                    # Series already complete
                    winner = team1 if score1 > score2 else team2
                else:  # in_progress
                    # Simulate remaining games
                    winner = simulate_in_progress_series(team1, team2, score1, score2, model)
                
                # Store winner of this matchup
                r1_winners['West'][seeds] = winner
            
            # Simulate remaining Eastern Conference first round
            for seeds, (team1, team2, score1, score2, status) in current_status['east_r1'].items():
                if status == 'complete':
                    # Series already complete
                    winner = team1 if score1 > score2 else team2
                else:  # in_progress
                    # Simulate remaining games
                    winner = simulate_in_progress_series(team1, team2, score1, score2, model)
                
                # Store winner of this matchup
                r1_winners['East'][seeds] = winner
            
            # Create second round matchups (based on standard NBA bracket)
            # Western Conference
            west_r2 = [
                # 1/8 winner vs 4/5 winner
                (r1_winners['West'][(1, 8)], r1_winners['West'][(4, 5)]),
                # 2/7 winner vs 3/6 winner
                (r1_winners['West'][(2, 7)], r1_winners['West'][(3, 6)])
            ]
            
            # Eastern Conference
            east_r2 = [
                # 1/8 winner vs 4/5 winner
                (r1_winners['East'][(1, 8)], r1_winners['East'][(4, 5)]),
                # 2/7 winner vs 3/6 winner
                (r1_winners['East'][(2, 7)], r1_winners['East'][(3, 6)])
            ]
            
            # Simulate second round
            west_r2_winners = []
            for team1, team2 in west_r2:
                if team1 and team2:  # Ensure both teams are valid
                    p_series = get_series_prob(team1, team2, model)
                    winner = team1 if random.random() < p_series else team2
                    west_r2_winners.append(winner)
            
            east_r2_winners = []
            for team1, team2 in east_r2:
                if team1 and team2:  # Ensure both teams are valid
                    p_series = get_series_prob(team1, team2, model)
                    winner = team1 if random.random() < p_series else team2
                    east_r2_winners.append(winner)
            
            # Conference Finals
            if len(west_r2_winners) == 2:
                team1, team2 = west_r2_winners
                p_series = get_series_prob(team1, team2, model)
                west_winner = team1 if random.random() < p_series else team2
                # Record conference champion
                conf_champs['West'][west_winner] += 1
            else:
                # Default (shouldn't happen)
                west_winner = 'OKC'
                conf_champs['West'][west_winner] += 1
            
            if len(east_r2_winners) == 2:
                team1, team2 = east_r2_winners
                p_series = get_series_prob(team1, team2, model)
                east_winner = team1 if random.random() < p_series else team2
                # Record conference champion
                conf_champs['East'][east_winner] += 1
            else:
                # Default (shouldn't happen)
                east_winner = 'BOS'
                conf_champs['East'][east_winner] += 1
            
            # NBA Finals
            finals_apps[east_winner] += 1
            finals_apps[west_winner] += 1
            
            p_series = get_series_prob(east_winner, west_winner, model)
            champion = east_winner if random.random() < p_series else west_winner
            
            # Record champion
            champion_counts[champion] += 1
        
        # Convert to probabilities for this model
        all_results[model] = {
            'champion_probs': {team: count/n_simulations for team, count in champion_counts.items() if count > 0},
            'conf_champ_probs': {
                'West': {team: count/n_simulations for team, count in conf_champs['West'].items() if count > 0},
                'East': {team: count/n_simulations for team, count in conf_champs['East'].items() if count > 0}
            },
            'finals_probs': {team: count/n_simulations for team, count in finals_apps.items() if count > 0}
        }
        
        # Display top teams for this model
        print(f"\n{model} Model - Championship Odds:")
        for team, prob in sorted(all_results[model]['champion_probs'].items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"{team}: {prob:.1%}")
    
    # Create ensemble by averaging across models
    ensemble_results = {
        'champion_probs': {},
        'conf_champ_probs': {'West': {}, 'East': {}},
        'finals_probs': {}
    }
    
    # Get all teams from individual model results
    all_champ_teams = set()
    all_finals_teams = set()
    all_west_teams = set()
    all_east_teams = set()
    
    for model in all_results:
        all_champ_teams.update(all_results[model]['champion_probs'].keys())
        all_finals_teams.update(all_results[model]['finals_probs'].keys())
        all_west_teams.update(all_results[model]['conf_champ_probs']['West'].keys())
        all_east_teams.update(all_results[model]['conf_champ_probs']['East'].keys())
    
    # Initialize with zeros
    for team in all_champ_teams:
        ensemble_results['champion_probs'][team] = 0
    
    for team in all_finals_teams:
        ensemble_results['finals_probs'][team] = 0
    
    for team in all_west_teams:
        ensemble_results['conf_champ_probs']['West'][team] = 0
    
    for team in all_east_teams:
        ensemble_results['conf_champ_probs']['East'][team] = 0
    
    # Average the probabilities
    for model in all_results:
        for team in all_results[model]['champion_probs']:
            ensemble_results['champion_probs'][team] += all_results[model]['champion_probs'][team] / len(all_results)
        
        for team in all_results[model]['finals_probs']:
            ensemble_results['finals_probs'][team] += all_results[model]['finals_probs'][team] / len(all_results)
        
        for team in all_results[model]['conf_champ_probs']['West']:
            ensemble_results['conf_champ_probs']['West'][team] += all_results[model]['conf_champ_probs']['West'][team] / len(all_results)
        
        for team in all_results[model]['conf_champ_probs']['East']:
            ensemble_results['conf_champ_probs']['East'][team] += all_results[model]['conf_champ_probs']['East'][team] / len(all_results)
    
    return {'ensemble': ensemble_results, 'models': all_results}

# Run the simulation
print("\nStarting playoff simulations...")
n_sims = 10000
results = simulate_remaining_playoffs(n_sims)

# Display the ensemble results
print("\n--- ENSEMBLE MODEL PLAYOFF PREDICTIONS ---")
print("\nNBA Championship Probability:")
for team, prob in sorted(results['ensemble']['champion_probs'].items(), key=lambda x: x[1], reverse=True):
    if prob > 0.01:  # Only show teams with >1% chance
        print(f"{team}: {prob:.1%}")

print("\nConference Champions Probability:")
for conf in ['East', 'West']:
    print(f"\n{conf}ern Conference Champion:")
    for team, prob in sorted(results['ensemble']['conf_champ_probs'][conf].items(), key=lambda x: x[1], reverse=True):
        if prob > 0.01:  # Only show teams with >1% chance
            print(f"{team}: {prob:.1%}")

print("\nNBA Finals Appearance Probability:")
for team, prob in sorted(results['ensemble']['finals_probs'].items(), key=lambda x: x[1], reverse=True):
    if prob > 0.01:  # Only show teams with >1% chance
        print(f"{team}: {prob:.1%}")

Current NBA Playoff Status:

Western Conference - First Round:
(1) OKC def. (8) MEM, 4-0
(4) DEN leads (5) LAC, 3-2
(3) LAL def. (6) MIN, 1-4
(2) HOU leads (7) GSW, 3-2

Eastern Conference - First Round:
(1) CLE def. (8) MIA, 4-0
(4) IND def. (5) MIL, 4-1
(3) NYK leads (6) DET, 3-2
(2) BOS def. (7) ORL, 4-1

Starting playoff simulations...

Running 10000 simulations using BT_ability model...

BT_ability Model - Championship Odds:
OKC: 32.8%
CLE: 21.3%
BOS: 16.2%
HOU: 6.8%
MIN: 5.9%

Running 10000 simulations using Elo_rating model...

Elo_rating Model - Championship Odds:
OKC: 53.1%
CLE: 25.3%
BOS: 14.8%
HOU: 2.3%
NYK: 2.0%

Running 10000 simulations using Win_pct model...

Win_pct Model - Championship Odds:
OKC: 21.7%
CLE: 18.0%
BOS: 16.5%
MIN: 8.6%
HOU: 8.2%

--- ENSEMBLE MODEL PLAYOFF PREDICTIONS ---

NBA Championship Probability:
OKC: 35.9%
CLE: 21.5%
BOS: 15.8%
HOU: 5.8%
MIN: 5.0%
NYK: 4.6%
IND: 4.6%
DEN: 3.7%
LAC: 1.2%
GSW: 1.1%

Conference Champions Probability:

Eastern Confere

In [9]:
print("Step 7: Compute Win-% Prior")
print("===========================")

# Assuming team_ratings is already available from your previous code
# It contains the Win_pct field that we'll use as our Win-% Prior

# Make a copy to avoid modifying the original
win_pct_prior = team_ratings['Win_pct'].copy()

# Rescale win percentages to sum to 1 (turn into probability distribution)
win_pct_prior = win_pct_prior / win_pct_prior.sum()

print("\nWin Percentage Prior (Top 10 teams):")
top_win_pct = win_pct_prior.sort_values(ascending=False).head(10)
for team, prob in top_win_pct.items():
    print(f"{team}: {prob:.3f}")


Step 7: Compute Win-% Prior

Win Percentage Prior (Top 10 teams):
OKC: 0.055
CLE: 0.052
BOS: 0.050
HOU: 0.042
NYK: 0.041
LAC: 0.041
DEN: 0.041
LAL: 0.041
IND: 0.041
MIN: 0.040


In [10]:
df_betting.head(10)

,Outcome,Implied Probability (%),Market Description,Data Source,ImpProb,Overround,NormProb
0,Oklahoma City Thunder,38.0,Championship Winner,Polymarket,0.38,1.02,0.372549
1,Boston Celtics,29.0,Championship Winner,Polymarket,0.29,1.02,0.284314
2,Cleveland Cavaliers,16.0,Championship Winner,Polymarket,0.16,1.02,0.156863
3,Golden State Warriors,6.0,Championship Winner,Polymarket,0.06,1.02,0.058824
4,Minnesota Timberwolves,4.0,Championship Winner,Polymarket,0.04,1.02,0.039216
5,Denver Nuggets,3.0,Championship Winner,Polymarket,0.03,1.02,0.029412
6,Los Angeles Lakers,2.0,Championship Winner,Polymarket,0.02,1.02,0.019608
7,Los Angeles Clippers,2.0,Championship Winner,Polymarket,0.02,1.02,0.019608
8,New York Knicks,1.0,Championship Winner,Polymarket,0.01,1.02,0.009804
9,Indiana Pacers,1.0,Championship Winner,Polymarket,0.01,1.02,0.009804


In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
import random
from math import comb
import os
import traceback

# For reproducibility
random.seed(42)
np.random.seed(42)

# --- Configuration ---
HISTORICAL_YEARS = [2023, 2024]
# CURRENT_FORECAST_YEAR = 2025 # Removed - Not needed for backtesting only

# Actual champions for historical seasons for Brier score calculation
ACTUAL_CHAMPIONS = {
    2023: 'DEN',  # Denver Nuggets
    2024: 'BOS'   # Boston Celtics
}

# Playoff Seeds Data for Historical Years Only
PLAYOFF_SEEDS_DATA = {
    2023: { # Actual 2023 seeds
        'West': {1: 'DEN', 2: 'MEM', 3: 'SAC', 4: 'PHX', 5: 'LAC', 6: 'GSW', 7: 'LAL', 8: 'MIN'},
        'East': {1: 'MIL', 2: 'BOS', 3: 'PHI', 4: 'CLE', 5: 'NYK', 6: 'BKN', 7: 'MIA', 8: 'ATL'}
    },
    2024: { # Actual 2024 seeds
        'West': {1: 'OKC', 2: 'DEN', 3: 'MIN', 4: 'LAC', 5: 'DAL', 6: 'PHX', 7: 'NOP', 8: 'LAL'},
        'East': {1: 'BOS', 2: 'NYK', 3: 'MIL', 4: 'CLE', 5: 'ORL', 6: 'IND', 7: 'PHI', 8: 'MIA'}
    }
    # Removed 2025 seeds as we are only backtesting
}

# --- TEAM ABBREVIATION MAPPING ---
# This should be populated reliably. Example provided.
TEAM_NAME_TO_ABBR_MAP_GLOBAL = {
    'Denver Nuggets': 'DEN', 'Boston Celtics': 'BOS', 'Miami Heat': 'MIA',
    'Los Angeles Lakers': 'LAL', 'Phoenix Suns': 'PHX', 'Milwaukee Bucks': 'MIL',
    'Golden State Warriors': 'GSW', 'Dallas Mavericks': 'DAL', 'Minnesota Timberwolves': 'MIN',
    'Cleveland Cavaliers': 'CLE', 'Philadelphia 76ers': 'PHI', 'Memphis Grizzlies': 'MEM',
    'Sacramento Kings': 'SAC', 'New York Knicks': 'NYK', 'Los Angeles Clippers': 'LAC',
    'Oklahoma City Thunder': 'OKC', 'Brooklyn Nets': 'BKN', 'Atlanta Hawks': 'ATL',
    'Chicago Bulls': 'CHI', 'New Orleans Pelicans': 'NOP', 'Orlando Magic': 'ORL',
    'Indiana Pacers': 'IND', 'Toronto Raptors': 'TOR', 'Washington Wizards': 'WAS',
    'Utah Jazz': 'UTA', 'Houston Rockets': 'HOU', 'Detroit Pistons': 'DET',
    'Charlotte Hornets': 'CHA', 'San Antonio Spurs': 'SAS', 'Portland Trail Blazers': 'POR'
    # Add others if necessary based on your data sources
}

# --- Brier Score Function ---
def brier_score(predictions: pd.Series, actual_winner: str) -> float:
    """Computes the Brier score for probabilistic forecasts."""
    score = 0.0
    predictions_temp = predictions.copy() # Avoid modifying original Series
    if actual_winner not in predictions_temp.index:
        predictions_temp[actual_winner] = 0.0 # Add winner with 0 prob if missing

    # Include teams in the prediction that might not have won
    all_predicted_teams = predictions_temp.index
    for team in all_predicted_teams:
        prob = predictions_temp.get(team, 0.0) # Use get for safety
        outcome = 1.0 if team == actual_winner else 0.0
        score += (prob - outcome) ** 2

    # If the actual winner wasn't even in the original predictions, add their term
    if actual_winner not in predictions.index:
         outcome = 1.0
         prob = 0.0 # They were assigned 0 probability initially
         score += (prob - outcome)**2

    return score

# --- Function to Process One Historical Season ---
# (Includes H2H processing, model building, prior generation)
def process_historical_season(year: int, team_name_to_abbr_map: dict, playoff_seeds_for_year: dict):
    print(f"\nProcessing historical data for {year} season...")
    h2h_file = f'teamVsteam{year}.csv'
    # Fix the betting file path - this was likely causing issues
    betting_file_path = f'data/bettingmarkets/betting_data_{year}.csv' 
    
    # Try alternate paths if the betting file is not found
    if not os.path.exists(betting_file_path):
        alternate_paths = [
            f'data/bettingmarkets/betting_data_{year}.csv',
            f'betting_data_{year}.csv'
        ]
        for alt_path in alternate_paths:
            if os.path.exists(alt_path):
                betting_file_path = alt_path
                print(f"Found betting data at alternate path: {betting_file_path}")
                break
    
    if not os.path.exists(h2h_file):
        print(f"Error: H2H file {h2h_file} not found. Skipping year {year}.")
        return None
    
    print(f"Loading head-to-head data from {h2h_file}")
    df_h2h_hist = pd.read_csv(h2h_file)

    # --- H2H Parsing & Abbreviation Mapping ---
    teams = []
    local_abbr_map = {}
    if 'Team' in df_h2h_hist.columns:
        for full_name in df_h2h_hist['Team']:
            abbr = team_name_to_abbr_map.get(full_name)
            if not abbr:
                abbr = full_name[:3].upper() + str(random.randint(10,99))
                print(f"Warning: Used fallback abbreviation '{abbr}' for '{full_name}' in {year} H2H.")
                team_name_to_abbr_map[full_name] = abbr # Update global map too
            local_abbr_map[full_name] = abbr
            teams.append(abbr)
        teams = sorted(list(set(teams)))

        df_h2h_hist_indexed = df_h2h_hist.set_index('Team')
        if 'Rk' in df_h2h_hist_indexed.columns: df_h2h_hist_indexed = df_h2h_hist_indexed.drop(columns=['Rk'])
        df_h2h_hist_indexed.index = df_h2h_hist_indexed.index.map(local_abbr_map)
        
        # Fix issue with column mapping
        # First, get the original column names
        original_columns = df_h2h_hist_indexed.columns.tolist()
        mapped_columns = []
        
        # Map only the team name columns, not numeric/other columns
        for col in original_columns:
            if col in local_abbr_map:
                mapped_columns.append(local_abbr_map[col])
            else:
                mapped_columns.append(col)
        
        df_h2h_hist_indexed.columns = mapped_columns
    else:
        print(f"Error: 'Team' column missing in {h2h_file}. Skipping year {year}.")
        return None

    if not teams:
        print(f"Error: No teams identified for {year}. Skipping year {year}.")
        return None

    # --- Expand H2H to Matches ---
    matches = []
    # ...(Same H2H expansion logic as previous version)...
    for i, team_i in enumerate(teams):
        for j, team_j in enumerate(teams):
            if i >= j or team_i not in df_h2h_hist_indexed.index or team_j not in df_h2h_hist_indexed.columns:
                continue
            record_val = df_h2h_hist_indexed.at[team_i, team_j]
            if pd.isna(record_val) or record_val == '': continue
            try:
                wins, losses = map(int, str(record_val).split('-'))
                matches.extend([{'winner': team_i, 'loser': team_j}] * wins)
                matches.extend([{'winner': team_j, 'loser': team_i}] * losses)
            except (ValueError, AttributeError):
                 print(f"Warning: Could not parse H2H record '{record_val}' for {team_i} vs {team_j}, year {year}")
    df_matches_hist = pd.DataFrame(matches)

    print(f"Created {len(df_matches_hist)} match records for {year}")
    if df_matches_hist.empty:
        print(f"Warning: No matches generated for {year}. Models cannot be built.")
        return {'year': year, 'market_prior': pd.Series(dtype=float), 'win_pct_prior': pd.Series(dtype=float),
                'simulation_prior': pd.Series(dtype=float), 'actual_champion': ACTUAL_CHAMPIONS.get(year)}

    # --- Build Models (BT, Elo, Win%) ---
    team_indices = {team: i for i, team in enumerate(teams)}
    n_teams = len(teams)
    X = np.zeros((len(df_matches_hist), n_teams))
    valid_matches_count = 0
    # ...(Same BT matrix setup logic as previous version)...
    for i, row in df_matches_hist.iterrows():
        winner_idx = team_indices.get(row['winner'])
        loser_idx = team_indices.get(row['loser'])
        if winner_idx is not None and loser_idx is not None:
            X[valid_matches_count, winner_idx] = 1
            X[valid_matches_count, loser_idx] = -1
            valid_matches_count += 1
    
    if valid_matches_count == 0: 
        print(f"No valid matches found for {year}")
        return None # Cannot proceed
    
    X = X[:valid_matches_count]
    y = np.ones(valid_matches_count)
    
    print(f"Training Bradley-Terry model for {year}...")
    bt_model = Ridge(alpha=0.001, fit_intercept=False).fit(X, y)
    raw_abilities = pd.Series(bt_model.coef_, index=teams)
    min_ab, max_ab = raw_abilities.min(), raw_abilities.max()
    scaled_abilities = (raw_abilities - min_ab) * 100 / (max_ab - min_ab) if (max_ab - min_ab) != 0 else pd.Series(50.0, index=teams)

    print(f"Calculating win percentages for {year}...")
    win_counts = df_matches_hist['winner'].value_counts()
    loss_counts = df_matches_hist['loser'].value_counts()
    win_pct_vals = {t: win_counts.get(t,0)/(win_counts.get(t,0)+loss_counts.get(t,0)) if (win_counts.get(t,0)+loss_counts.get(t,0)) > 0 else 0.0 for t in teams}
    win_pct_series = pd.Series(win_pct_vals).reindex(teams).fillna(0.0)

    print(f"Calculating Elo ratings for {year}...")
    elo_ratings = {t: 1500 for t in teams}
    K=20
    # ...(Same Elo calculation logic as previous version)...
    for _, row in df_matches_hist.iterrows():
        w, l = row['winner'], row['loser']
        if w not in elo_ratings or l not in elo_ratings: continue
        Rw, Rl = elo_ratings[w], elo_ratings[l]
        Ew = 1 / (1 + 10**((Rl - Rw) / 400))
        elo_ratings[w] += K * (1 - Ew)
        elo_ratings[l] += K * (0 - Ew) # Correct Elo update for loser
    elo_series = pd.Series(elo_ratings).reindex(teams).fillna(1500)

    team_ratings_hist = pd.DataFrame({
        'BT_ability_scaled': scaled_abilities, 'BT_ability_raw': raw_abilities,
        'Elo_rating': elo_series, 'Win_pct': win_pct_series
    }).reindex(teams)

      # --- Market Prior M_T ---
    market_prior_hist = pd.Series(dtype=float)
    print(f"Checking for betting data at {betting_file_path}")

    # Hard-coded betting data for 2023 and 2024
    BETTING_DATA_2023 = {
        'DEN': 0.091,  # Denver Nuggets
        'MIA': 0.074,  # Miami Heat
        'BOS': 0.357,  # Boston Celtics
        'LAL': 0.111,  # Los Angeles Lakers
        'PHX': 0.200,  # Phoenix Suns
        'GSW': 0.167,  # Golden State Warriors
        'PHI': 0.143,  # Philadelphia 76ers
        'NYK': 0.048,  # New York Knicks
        'MIL': 0.267,  # Milwaukee Bucks
        'MEM': 0.053,  # Memphis Grizzlies
        'CLE': 0.024,  # Cleveland Cavaliers
        'LAC': 0.020,  # Los Angeles Clippers
        'SAC': 0.020,  # Sacramento Kings
        'MIN': 0.007,  # Minnesota Timberwolves
        'ATL': 0.005,  # Atlanta Hawks
        'BKN': 0.002   # Brooklyn Nets
    }

    BETTING_DATA_2024 = {
        'BOS': 0.385,  # Boston Celtics
        'DAL': 0.050,  # Dallas Mavericks
        'MIN': 0.048,  # Minnesota Timberwolves
        'IND': 0.010,  # Indiana Pacers
        'DEN': 0.256,  # Denver Nuggets
        'OKC': 0.059,  # Oklahoma City Thunder
        'NYK': 0.024,  # New York Knicks
        'CLE': 0.020,  # Cleveland Cavaliers
        'LAC': 0.071,  # Los Angeles Clippers
        'MIL': 0.067,  # Milwaukee Bucks
        'PHI': 0.067,  # Philadelphia 76ers
        'PHX': 0.067,  # Phoenix Suns
        'LAL': 0.038,  # Los Angeles Lakers
        'MIA': 0.024,  # Miami Heat
        'NOP': 0.020,  # New Orleans Pelicans
        'ORL': 0.005   # Orlando Magic
    }

    # Use the appropriate hard-coded data based on the year
    if year == 2023:
        print(f"Using hard-coded 2023 betting data")
        market_data_raw = BETTING_DATA_2023
    elif year == 2024:
        print(f"Using hard-coded 2024 betting data")
        market_data_raw = BETTING_DATA_2024
    else:
        # For any other year, try to load from file
        market_data_raw = {}
        if os.path.exists(betting_file_path):
            try:
                # Load betting data
                print(f"Loading betting data from {betting_file_path}")
                df_betting_hist_raw = pd.read_csv(betting_file_path)
                
                # Continue with standard processing for non-hardcoded years
                # [... rest of the existing betting data processing code ...]
            except Exception as e:
                print(f"Error processing betting data file: {e}")
                market_data_raw = {}
        else:
            print(f"Betting data file not found at {betting_file_path}")

    # If we have hard-coded data, convert to Series
    if market_data_raw and year in [2023, 2024]:
        market_prior_hist = pd.Series(market_data_raw)
        
        # Calculate the sum to check normalization
        market_sum = market_prior_hist.sum()
        print(f"Sum of market probabilities for {year}: {market_sum:.3f}")
        
        # Normalize if needed
        if not np.isclose(market_sum, 1.0, atol=0.01):
            print(f"Normalizing market prior (sum was {market_sum:.3f})")
            market_prior_hist = market_prior_hist / market_sum
        
        # Debug the market prior
        print(f"Market prior for {year} (top 5):")
        top_5_market = market_prior_hist.sort_values(ascending=False).head(5)
        for team, prob in top_5_market.items():
            print(f"  {team}: {prob:.3f}")
    
        
    # --- Create synthetic market prior if needed ---
    if market_prior_hist.empty:
        print(f"Creating synthetic market prior for {year}...")
        current_year_playoff_teams = set()
        for conf in playoff_seeds_for_year.values():
            for team in conf.values():
                current_year_playoff_teams.add(team)
        
        current_year_playoff_teams = list(current_year_playoff_teams)
        
        if current_year_playoff_teams:
            # Create a synthetic prior based on seed positions
            seed_based_market_prior = {}
            seed_weights = {1: 4.0, 2: 3.0, 3: 2.5, 4: 2.0, 5: 1.5, 6: 1.0, 7: 0.75, 8: 0.5}
            
            for conf_name, conf_seeds in playoff_seeds_for_year.items():
                for seed_num, team_abbr in conf_seeds.items():
                    if team_abbr in seed_based_market_prior:
                        seed_based_market_prior[team_abbr] += seed_weights.get(seed_num, 1.0)
                    else:
                        seed_based_market_prior[team_abbr] = seed_weights.get(seed_num, 1.0)
            
            # Convert to Series and normalize
            market_prior_hist = pd.Series(seed_based_market_prior)
            market_prior_hist = market_prior_hist / market_prior_hist.sum()
            
            print(f"Created seed-based synthetic market prior (top 5):")
            top_5_synthetic = market_prior_hist.sort_values(ascending=False).head(5)
            for team, prob in top_5_synthetic.items():
                print(f"  {team}: {prob:.3f}")
        else:
            print(f"Warning: No playoff teams defined for {year}, cannot create synthetic market prior")
            market_prior_hist = pd.Series(dtype=float)

    # --- Win Percentage Prior W_T ---
    print(f"Creating win percentage prior for {year}...")
    wp_raw = team_ratings_hist['Win_pct'].copy()
    wp_raw[wp_raw <= 0] = 1e-9
    win_pct_prior_hist = wp_raw / wp_raw.sum() if wp_raw.sum() > 0 else pd.Series(dtype=float)

    # --- Simulation Prior S_T ---
    # (Reusing the simulation logic defined previously)
    def get_game_prob_hist_sim(t1, t2, model_key, ratings_df):
        if t1 not in ratings_df.index or t2 not in ratings_df.index:
            s1_seed = [s for s_conf, t_conf in playoff_seeds_for_year['West'].items() if t_conf == t1] + [s for s_conf, t_conf in playoff_seeds_for_year['East'].items() if t_conf == t1]
            s2_seed = [s for s_conf, t_conf in playoff_seeds_for_year['West'].items() if t_conf == t2] + [s for s_conf, t_conf in playoff_seeds_for_year['East'].items() if t_conf == t2]
            if s1_seed and s2_seed: return min(1, max(0, 0.5 + 0.025 * (s2_seed[0] - s1_seed[0])))
            return 0.5
        if model_key == 'BT_ability_raw':
            diff = ratings_df.loc[t1, 'BT_ability_raw'] - ratings_df.loc[t2, 'BT_ability_raw']
            return 1 / (1 + np.exp(-diff))
        elif model_key == 'Elo_rating':
            R1, R2 = ratings_df.loc[t1, 'Elo_rating'], ratings_df.loc[t2, 'Elo_rating']
            return 1 / (1 + 10**((R2 - R1) / 400))
        else: # Win_pct
            w1, w2 = ratings_df.loc[t1, 'Win_pct'], ratings_df.loc[t2, 'Win_pct']
            w1_adj, w2_adj = 0.5 + (w1 - 0.5) * 0.67, 0.5 + (w2 - 0.5) * 0.67
            return w1_adj / (w1_adj + w2_adj) if (w1_adj + w2_adj) > 0 else 0.5

    def get_series_prob_hist_sim(t1, t2, model_key_loc, ratings_df_loc):
        p_game = get_game_prob_hist_sim(t1, t2, model_key_loc, ratings_df_loc)
        if p_game <= 0.0: return 0.0
        if p_game >= 1.0: return 1.0
        q_game = 1.0 - p_game; total_prob = 0.0
        for k_losses_loc in range(4): total_prob += comb(3 + k_losses_loc, k_losses_loc) * (p_game**4) * (q_game**k_losses_loc)
        return total_prob

    print(f"Running playoff simulations for {year}...")
    temp_sim_priors_all_models = {}
    sim_models_to_use = ['BT_ability_raw', 'Elo_rating', 'Win_pct']
    playoff_teams_for_sim = []
    for conf_dict in playoff_seeds_for_year.values():
        playoff_teams_for_sim.extend(conf_dict.values())
    
    if not playoff_teams_for_sim:
         print(f"Error: No playoff teams defined for simulation in year {year}. Skipping simulation.")
         simulation_prior_hist = pd.Series(dtype=float)
    else:
        for model_type_sim in sim_models_to_use:
             # ...(Same simulation logic as previous version)...
            print(f"Running simulations with {model_type_sim} model...")
            current_model_champ_counts = {team_code: 0 for team_code in playoff_teams_for_sim}
            n_sims_hist = 5000  # Increased for better accuracy
            for _ in range(n_sims_hist):
                # West
                west_r1w = {}
                for s1, s2 in [(1,8), (4,5), (3,6), (2,7)]:
                    if s1 in playoff_seeds_for_year['West'] and s2 in playoff_seeds_for_year['West']:
                        team1 = playoff_seeds_for_year['West'][s1]
                        team2 = playoff_seeds_for_year['West'][s2]
                        p_series = get_series_prob_hist_sim(team1, team2, model_type_sim, team_ratings_hist)
                        west_r1w[s1] = team1 if random.random() < p_series else team2
                
                if len(west_r1w) < 4: continue  # Skip this simulation if bracket incomplete
                
                w_sf1_w = west_r1w[1] if random.random() < get_series_prob_hist_sim(west_r1w[1], west_r1w[4], model_type_sim, team_ratings_hist) else west_r1w[4]
                w_sf2_w = west_r1w[3] if random.random() < get_series_prob_hist_sim(west_r1w[3], west_r1w[2], model_type_sim, team_ratings_hist) else west_r1w[2]
                west_champ = w_sf1_w if random.random() < get_series_prob_hist_sim(w_sf1_w, w_sf2_w, model_type_sim, team_ratings_hist) else w_sf2_w
                
                # East
                east_r1w = {}
                for s1, s2 in [(1,8), (4,5), (3,6), (2,7)]:
                    if s1 in playoff_seeds_for_year['East'] and s2 in playoff_seeds_for_year['East']:
                        team1 = playoff_seeds_for_year['East'][s1]
                        team2 = playoff_seeds_for_year['East'][s2]
                        p_series = get_series_prob_hist_sim(team1, team2, model_type_sim, team_ratings_hist)
                        east_r1w[s1] = team1 if random.random() < p_series else team2
                
                if len(east_r1w) < 4: continue  # Skip this simulation if bracket incomplete
                
                e_sf1_w = east_r1w[1] if random.random() < get_series_prob_hist_sim(east_r1w[1], east_r1w[4], model_type_sim, team_ratings_hist) else east_r1w[4]
                e_sf2_w = east_r1w[3] if random.random() < get_series_prob_hist_sim(east_r1w[3], east_r1w[2], model_type_sim, team_ratings_hist) else east_r1w[2]
                east_champ = e_sf1_w if random.random() < get_series_prob_hist_sim(e_sf1_w, e_sf2_w, model_type_sim, team_ratings_hist) else e_sf2_w
                
                # Finals
                nba_champ = east_champ if random.random() < get_series_prob_hist_sim(east_champ, west_champ, model_type_sim, team_ratings_hist) else west_champ
                current_model_champ_counts[nba_champ] += 1
            
            # Convert counts to probabilities
            temp_sim_priors_all_models[model_type_sim] = {k: v / n_sims_hist for k, v in current_model_champ_counts.items()}

        # Average across models
        simulation_prior_hist_dict = {}
        for team_code_val in playoff_teams_for_sim:
            model_probs = [temp_sim_priors_all_models[m_key].get(team_code_val, 0) for m_key in sim_models_to_use]
            simulation_prior_hist_dict[team_code_val] = np.mean(model_probs)
        
        # Convert to Series and normalize
        simulation_prior_hist = pd.Series(simulation_prior_hist_dict)
        if simulation_prior_hist.sum() > 0:
            simulation_prior_hist = simulation_prior_hist / simulation_prior_hist.sum()
            
            # Debug simulation prior
            print(f"Simulation prior for {year} (top 5):")
            top_5_sim = simulation_prior_hist.sort_values(ascending=False).head(5)
            for team, prob in top_5_sim.items():
                print(f"  {team}: {prob:.3f}")
        else:
            print(f"Warning: Simulation prior for {year} sums to 0.")
            simulation_prior_hist = pd.Series(dtype=float)


    print(f"\n--- Priors Generated for {year} ---")
    # Return the results as a dictionary
    return {
        'year': year,
        'market_prior': market_prior_hist.reindex(teams, fill_value=0.0), 
        'win_pct_prior': win_pct_prior_hist.reindex(teams, fill_value=0.0),
        'simulation_prior': simulation_prior_hist.reindex(teams, fill_value=0.0),
        'actual_champion': ACTUAL_CHAMPIONS.get(year)
    }

def apply_multiplication(market_p: pd.Series, win_pct_p: pd.Series, simulation_p: pd.Series, exponents: dict) -> pd.Series:
    all_teams = list(set(market_p.index.tolist() + win_pct_p.index.tolist() + simulation_p.index.tolist()))
    if not all_teams: return pd.Series(dtype=float)
    result = pd.Series(index=all_teams, dtype=float)
    default_prob_floor = 1e-9
    for team_code in all_teams:
        m_prob = max(market_p.get(team_code, 0.0), default_prob_floor)
        w_prob = max(win_pct_p.get(team_code, 0.0), default_prob_floor)
        s_prob_val = simulation_p.get(team_code, 0.0) # Get the actual sim prob
        s_prob_eff = max(s_prob_val, default_prob_floor) if s_prob_val > 0 else default_prob_floor
        result[team_code] = (m_prob ** exponents['Market']) * (w_prob ** exponents['Win_Pct']) * (s_prob_eff ** exponents['Simulation'])
    return result / result.sum() if result.sum() > 0 else result

def apply_weighted_average(market_p: pd.Series, win_pct_p: pd.Series, simulation_p: pd.Series, weights: dict) -> pd.Series:
    all_teams = list(set(market_p.index.tolist() + win_pct_p.index.tolist() + simulation_p.index.tolist()))
    if not all_teams: return pd.Series(dtype=float)
    result = pd.Series(index=all_teams, dtype=float)
    default_prob_floor = 0.0
    for team_code in all_teams:
        m_prob = market_p.get(team_code, default_prob_floor)
        w_prob = win_pct_p.get(team_code, default_prob_floor)
        s_prob = simulation_p.get(team_code, default_prob_floor)
        result[team_code] = (weights['Market'] * m_prob) + (weights['Win_Pct'] * w_prob) + (weights['Simulation'] * s_prob)
    return result / result.sum() if result.sum() > 0 else result


# --- Main Backtesting Execution ---
print("Step 10: Backtesting Using Historical Data")
print("=========================================")

historical_run_results = {}
# Pass the global map by reference if process_historical_season needs to update it with fallbacks
team_map_ref = TEAM_NAME_TO_ABBR_MAP_GLOBAL.copy() # Work with a copy if needed
for year_hist in HISTORICAL_YEARS:
    if year_hist not in PLAYOFF_SEEDS_DATA:
        print(f"Warning: Playoff seeds for historical year {year_hist} not defined. Skipping.")
        continue
    # Pass the necessary configurations for the specific year
    season_data = process_historical_season(year_hist, team_map_ref, PLAYOFF_SEEDS_DATA[year_hist])
    if season_data and season_data['actual_champion']:
        # Check if priors are valid before storing
        has_market = not season_data['market_prior'].empty
        has_win_pct = not season_data['win_pct_prior'].empty
        has_sim = not season_data['simulation_prior'].empty
        
        print(f"Valid priors for {year_hist}: Market={has_market}, Win_Pct={has_win_pct}, Simulation={has_sim}")
        
        if has_market and has_win_pct and has_sim:
            historical_run_results[year_hist] = season_data
            print(f"Successfully processed year {year_hist}")
        else:
            print(f"Warning: Some priors missing or empty for year {year_hist}. Excluding from optimization.")
    else:
        print(f"Warning: Processing failed or no actual champion found for year {year_hist}.")

# --- Hyperparameter Optimization ---
best_method = 'multiplication' # Default
best_params = {'Market': 1.0, 'Win_Pct': 1.0, 'Simulation': 1.0} # Default exponents

if historical_run_results and len(historical_run_results) > 0:
    print(f"\nOptimizing hyperparameters based on {len(historical_run_results)} historical season(s)...")

    # Define grids for hyperparameter search
    mult_exponents = [ {'Market': a, 'Win_Pct': b, 'Simulation': c} for a in np.arange(0.2, 1.1, 0.2) for b in np.arange(0.2, 1.1, 0.2) for c in np.arange(0.2, 1.1, 0.2) ]
    weighted_weights_list = []
    for m_w in np.arange(0.1, 0.9, 0.1):
        for w_w in np.arange(0.1, 0.9 - m_w + 0.01, 0.1):
            s_w = 1.0 - m_w - w_w
            if s_w >= 0.09: weighted_weights_list.append({'Market': round(m_w,1), 'Win_Pct': round(w_w,1), 'Simulation': round(s_w,1)})

    mult_perf_results = []
    weighted_perf_results = []

    # --- Evaluate Multiplication Method ---
    print(f"\nTesting multiplication method with {len(mult_exponents)} exponent sets...")
    for exp_set in mult_exponents:
        total_brier = 0; valid_seasons_count = 0
        for year_data_key, data_val in historical_run_results.items():
            predictions = apply_multiplication(data_val['market_prior'], data_val['win_pct_prior'], data_val['simulation_prior'], exp_set)
            if not predictions.empty and data_val['actual_champion']:
                score = brier_score(predictions, data_val['actual_champion'])
                total_brier += score
                valid_seasons_count += 1
                print(f"Year {year_data_key}, exponents {exp_set}, Brier score: {score:.4f}")
        if valid_seasons_count > 0: 
            avg_score = total_brier / valid_seasons_count
            mult_perf_results.append({'exponents': exp_set, 'avg_brier_score': avg_score})
            print(f"Average Brier score for exponents {exp_set}: {avg_score:.4f}")

    # --- Evaluate Weighted Average Method ---
    print(f"\nTesting weighted average method with {len(weighted_weights_list)} weight sets...")
    for wt_set in weighted_weights_list:
        total_brier = 0; valid_seasons_count = 0
        for year_data_key, data_val in historical_run_results.items():
            predictions = apply_weighted_average(data_val['market_prior'], data_val['win_pct_prior'], data_val['simulation_prior'], wt_set)
            if not predictions.empty and data_val['actual_champion']:
                score = brier_score(predictions, data_val['actual_champion'])
                total_brier += score
                valid_seasons_count += 1
                print(f"Year {year_data_key}, weights {wt_set}, Brier score: {score:.4f}")
        if valid_seasons_count > 0: 
            avg_score = total_brier / valid_seasons_count
            weighted_perf_results.append({'weights': wt_set, 'avg_brier_score': avg_score})
            print(f"Average Brier score for weights {wt_set}: {avg_score:.4f}")

    # --- Determine Best Method and Parameters ---
    if mult_perf_results:
        best_mult = min(mult_perf_results, key=lambda x: x['avg_brier_score'])
    else:
        best_mult = {'exponents': best_params, 'avg_brier_score': float('inf')}
        
    if weighted_perf_results:
        best_weighted = min(weighted_perf_results, key=lambda x: x['avg_brier_score'])
    else:
        best_weighted = {'weights': best_params, 'avg_brier_score': float('inf')}

    print("\n--- Backtesting Results ---")
    if mult_perf_results: 
        print(f"Best Multiplication Exponents: {best_mult['exponents']}, Avg Brier Score: {best_mult['avg_brier_score']:.4f}")
    else: 
        print("Multiplication method yielded no valid results.")
        
    if weighted_perf_results: 
        print(f"Best Weighted Average Weights: {best_weighted['weights']}, Avg Brier Score: {best_weighted['avg_brier_score']:.4f}")
    else: 
        print("Weighted average method yielded no valid results.")

    if best_mult['avg_brier_score'] <= best_weighted['avg_brier_score'] and mult_perf_results:
        print("\nOptimal method based on backtesting: Multiplication")
        best_method = 'multiplication'
        best_params = best_mult['exponents']
    elif weighted_perf_results:
        print("\nOptimal method based on backtesting: Weighted Average")
        best_method = 'weighted'
        best_params = best_weighted['weights']
    else:
        print("\nNo valid optimization results found. Using default parameters (Multiplication, exponents=1.0).")
        # Defaults remain as initially set

else:
    print("\nNo valid historical results available for optimization. Using default parameters.")
    # Defaults remain as initially set

print(f"\n--- Backtesting Complete ---")
print(f"Recommended fusion method: {best_method}")
print(f"Recommended parameters: {best_params}")

# --- END OF SCRIPT ---
# The final prediction for 2025 would happen in a separate step/script,
# using the 'best_method' and 'best_params' found here, applied to
# the M_T, W_T, and S_T priors generated specifically for the 2025 season.

Step 10: Backtesting Using Historical Data

Processing historical data for 2023 season...
Loading head-to-head data from teamVsteam2023.csv
Created 1149 match records for 2023
Training Bradley-Terry model for 2023...
Calculating win percentages for 2023...
Calculating Elo ratings for 2023...
Checking for betting data at data/bettingmarkets/betting_data_2023.csv
Using hard-coded 2023 betting data
Sum of market probabilities for 2023: 1.589
Normalizing market prior (sum was 1.589)
Market prior for 2023 (top 5):
  BOS: 0.225
  MIL: 0.168
  PHX: 0.126
  GSW: 0.105
  PHI: 0.090
Creating win percentage prior for 2023...
Running playoff simulations for 2023...
Running simulations with BT_ability_raw model...
Running simulations with Elo_rating model...
Running simulations with Win_pct model...
Simulation prior for 2023 (top 5):
  DEN: 0.167
  BOS: 0.157
  MIL: 0.155
  PHI: 0.095
  MEM: 0.085

--- Priors Generated for 2023 ---
Valid priors for 2023: Market=True, Win_Pct=True, Simulation=True
S

In [12]:
# ----- Step 8: Bayesian Fusion of Priors & Simulation -----

print("Step 8: Bayesian Fusion of Priors & Simulation")
print("==============================================")

# Let's ensure we have all three priors in proper format
# 1. Market consensus prior - from betting data
try:
    # Filter betting data for championship winner market
    champion_market = df_betting[df_betting['Market Description'] == 'Championship Winner']
    
    # Create a Series with the market probabilities for championship
    # Using 'Outcome' column instead of 'Selection'
    market_prior = champion_market.set_index('Outcome')['NormProb']
    
    # Convert team names to abbreviations if needed
    # Create mapping dictionary from team names to abbreviations
    team_name_to_abbr = {
        'Oklahoma City Thunder': 'OKC',
        'Boston Celtics': 'BOS',
        'Cleveland Cavaliers': 'CLE',
        'Golden State Warriors': 'GSW',
        'Minnesota Timberwolves': 'MIN',
        'Denver Nuggets': 'DEN',
        'Los Angeles Lakers': 'LAL',
        'Los Angeles Clippers': 'LAC',
        'New York Knicks': 'NYK',
        'Indiana Pacers': 'IND',
        'Houston Rockets': 'HOU',
        'Memphis Grizzlies': 'MEM',
        'Miami Heat': 'MIA',
        'Milwaukee Bucks': 'MIL',
        'Detroit Pistons': 'DET',
        'Orlando Magic': 'ORL',
        'Philadelphia 76ers': 'PHI',
        'Phoenix Suns': 'PHX',
        'Dallas Mavericks': 'DAL'
        # Add any other teams that might appear in your data
    }
    
    # Create a new Series with abbreviated team names
    market_prior_abbr = pd.Series(index=team_ratings.index)
    
    # Transfer probabilities from full names to abbreviations
    for team_name, prob in market_prior.items():
        if team_name in team_name_to_abbr:
            abbr = team_name_to_abbr[team_name]
            if abbr in team_ratings.index:
                market_prior_abbr[abbr] = prob
    
    # Ensure all teams are represented
    for team in team_ratings.index:
        if pd.isna(market_prior_abbr[team]):
            market_prior_abbr[team] = 0.001  # Small baseline probability
    
    # Normalize to ensure probabilities sum to 1
    market_prior = market_prior_abbr / market_prior_abbr.sum()
    
    print("\nMarket consensus prior loaded successfully")
except Exception as e:
    print(f"\nWarning: Could not load market consensus prior: {e}")
    print("Creating synthetic market prior based on simulation results")
    
    # Create a synthetic prior if betting data unavailable
    market_prior = pd.Series(0, index=team_ratings.index)
    for team in team_ratings.index:
        if team in results['ensemble']['champion_probs']:
            market_prior[team] = results['ensemble']['champion_probs'][team]
        else:
            market_prior[team] = 0.001
    
    # Normalize
    market_prior = market_prior / market_prior.sum()

# 2. Win percentage prior
win_pct_prior = team_ratings['Win_pct'].copy()
win_pct_prior = win_pct_prior / win_pct_prior.sum()

# 3. Simulation-based frequencies 
simulation_prior = pd.Series(results['ensemble']['champion_probs'])

# Create a DataFrame with all three priors
priors_df = pd.DataFrame({
    'Market': market_prior,
    'Win_Pct': win_pct_prior,
    'Simulation': simulation_prior
})

# Fill NaN values with small values
priors_df = priors_df.fillna(0.001)

# Print the top teams according to each prior
print("\nTop teams by different priors:")
for prior_name in priors_df.columns:
    print(f"\n{prior_name} Prior (top 5):")
    top_teams = priors_df[prior_name].sort_values(ascending=False).head(5)
    for team, prob in top_teams.items():
        print(f"{team}: {prob:.3f}")

# ----- Apply Optimized Bayesian Fusion from Backtesting -----
print("\nApplying optimal fusion method from backtesting")

# Best method from backtesting: Multiplication
best_method = "multiplication"
# Best parameters from backtesting
best_params = {'Market': 0.6, 'Win_Pct': 1.0, 'Simulation': 1.0}

print(f"Best method: {best_method}")
print(f"Best parameters: {best_params}")

# Apply the multiplication method with optimized exponents
def apply_multiplication(priors, exponents):
    """Apply multiplication method with exponents."""
    # Apply exponents to each prior
    weighted_priors = pd.DataFrame(index=priors.index)
    for prior_name, exponent in exponents.items():
        # Use a minimum value to avoid zeros
        min_prob = 1e-9
        prior_values = priors[prior_name].copy()
        prior_values[prior_values < min_prob] = min_prob
        weighted_priors[prior_name] = prior_values ** exponent
    
    # Multiply all weighted priors
    unnormalized = weighted_priors.prod(axis=1)
    # Normalize
    return unnormalized / unnormalized.sum()

# Calculate optimal posterior using the multiplication method
optimized_posterior = apply_multiplication(priors_df, best_params)

# Print the final posterior probabilities
print("\nFinal championship probabilities using optimal parameters:")
for team, prob in optimized_posterior.sort_values(ascending=False).head(10).items():
    print(f"{team}: {prob:.3f} ({prob*100:.1f}%)")

# Calculate confidence intervals for the top teams
print("\nChampionship probabilities with 80% confidence intervals:")
top_teams = optimized_posterior.sort_values(ascending=False).head(5).index

for team in top_teams:
    p = optimized_posterior[team]
    # In a binomial distribution, the standard error is sqrt(p*(1-p)/n)
    # With n=10000 (approx. simulation count)
    se = np.sqrt(p * (1 - p) / 10000)
    
    # 80% confidence interval (using z=1.28 for 80% CI)
    lower = max(0, p - 1.28 * se)
    upper = min(1, p + 1.28 * se)
    
    print(f"{team}: {p:.3f} (80% CI: {lower:.3f} - {upper:.3f})")

# Store the final posterior for use in further analysis
final_probabilities = {
    'optimized': optimized_posterior,
    # Keep the original methods for comparison if desired
    'simple_multiplication': priors_df.prod(axis=1) / priors_df.prod(axis=1).sum(),
    'weighted_average': (0.4 * priors_df['Market'] + 0.2 * priors_df['Win_Pct'] + 
                         0.4 * priors_df['Simulation']) / 1.0
}

# Compare the optimized method with the original methods
print("\nComparison between optimized and original methods:")
comparison = pd.DataFrame({
    'Optimized': final_probabilities['optimized'],
    'Simple_Mult': final_probabilities['simple_multiplication'],
    'Weighted_Avg': final_probabilities['weighted_average']
}).fillna(0)

comparison = comparison.sort_values('Optimized', ascending=False).head(10)
print(comparison)

Step 8: Bayesian Fusion of Priors & Simulation

Market consensus prior loaded successfully

Top teams by different priors:

Market Prior (top 5):
OKC: 0.342
BOS: 0.296
CLE: 0.148
GSW: 0.052
MIN: 0.034

Win_Pct Prior (top 5):
OKC: 0.055
CLE: 0.052
BOS: 0.050
HOU: 0.042
NYK: 0.041

Simulation Prior (top 5):
OKC: 0.359
CLE: 0.215
BOS: 0.158
HOU: 0.058
MIN: 0.050

Applying optimal fusion method from backtesting
Best method: multiplication
Best parameters: {'Market': 0.6, 'Win_Pct': 1.0, 'Simulation': 1.0}

Final championship probabilities using optimal parameters:
OKC: 0.555 (55.5%)
BOS: 0.201 (20.1%)
CLE: 0.190 (19.0%)
MIN: 0.014 (1.4%)
NYK: 0.010 (1.0%)
IND: 0.009 (0.9%)
DEN: 0.008 (0.8%)
HOU: 0.005 (0.5%)
GSW: 0.004 (0.4%)
LAC: 0.003 (0.3%)

Championship probabilities with 80% confidence intervals:
OKC: 0.555 (80% CI: 0.549 - 0.561)
BOS: 0.201 (80% CI: 0.196 - 0.207)
CLE: 0.190 (80% CI: 0.185 - 0.195)
MIN: 0.014 (80% CI: 0.013 - 0.016)
NYK: 0.010 (80% CI: 0.009 - 0.012)

Comparison betw